# Question 122 - LC 295. Find Median from Data Stream

The **median** is the middle value in an ordered integer list. If the size of the list is even, there is no middle value, and the median is the mean of the two middle values.

- For example, for `arr = [2,3,4]`, the median is `3`.
- For example, for a`rr = [2,3]`, the median is `(2 + 3) / 2 = 2.5`.

Implement the MedianFinder class:

- `MedianFinder()` initializes the `MedianFinder` object.
- `void addNum(int num)` adds the integer `num` from the data stream to the data structure.
- `double findMedian()` returns the median of all elements so far. Answers within 10^-5 of the actual answer will be accepted.
 
## Example 1:

### Input

    ["MedianFinder", "addNum", "addNum", "findMedian", "addNum", "findMedian"]
    [[], [1], [2], [], [3], []]

### Output

    [null, null, null, 1.5, null, 2.0]

### Explanation

```java
MedianFinder medianFinder = new MedianFinder();
medianFinder.addNum(1);    // arr = [1]
medianFinder.addNum(2);    // arr = [1, 2]
medianFinder.findMedian(); // return 1.5 (i.e., (1 + 2) / 2)
medianFinder.addNum(3);    // arr[1, 2, 3]
medianFinder.findMedian(); // return 2.0
```

## Constraints:

- `-10^5 <= num <= 10^5`
- There will be at least one element in the data structure before calling `findMedian`.
- At most `5 * 10^4` calls will be made to `addNum` and `findMedian`.
 
## Follow up:

- If all integer numbers from the stream are in the range `[0, 100]`, how would you optimize your solution?
- If `99%` of all integer numbers from the stream are in the range `[0, 100]`, how would you optimize your solution?

In [11]:
import heapq as hq


class MedianFinder:

    def __init__(self):
        # max heap stores the smaller half of the numbers. Root of the heap is the largest number in the lower half
        self.lower_half = []  # max heap (negated)
        # min heap stores the larger half of the numbers. Root of the heap is the smallest number in the upper half
        self.upper_half = []  # min heap

    # Time complexity: O(log n)
    def addNum(self, num: int) -> None:
        hq.heappush(self.lower_half, -num)  # add to lower half

        # balance the heaps
        # the max heap can have at most 1 more element than the min heap
        # if the total number of elements is odd, the max heap will have 1 more element than the min heap

        if (
            self.lower_half
            and self.upper_half
            and -self.lower_half[0] > self.upper_half[0]
        ):
            # if the max heap has more elements than the min heap, move the root of the max heap to the min heap
            hq.heappush(self.upper_half, -hq.heappop(self.lower_half))

        # ensure that the max heap have at most 1 more element than the min heap
        if len(self.lower_half) > len(self.upper_half) + 1:
            hq.heappush(self.upper_half, -hq.heappop(self.lower_half))

        # ensure min heap has less elements than the max heap
        if len(self.upper_half) > len(self.lower_half):
            hq.heappush(self.lower_half, -hq.heappop(self.upper_half))

    # Time complexity: O(1)
    def findMedian(self) -> float:
        if len(self.lower_half) > len(self.upper_half):
            return float(-self.lower_half[0])
        return float((-self.lower_half[0] + self.upper_half[0]) / 2.0)

In [12]:
# Example usage
medianFinder = MedianFinder()
medianFinder.addNum(1)
medianFinder.addNum(2)
print(medianFinder.findMedian())  # Output: 1.5
medianFinder.addNum(3)
print(medianFinder.findMedian())  # Output: 2.0

1.5
2.0


### Optimizations for Specific Ranges:

1. **If all numbers are in the range [0, 100]**:
   - We can use counting sort techniques with a fixed-size array to store the frequency of each number.
   - This would allow for faster insertion and retrieval of the median, leveraging the fixed range of values.

2. **If 99% of numbers are in the range [0, 100]**:
   - Use a combination of counting sort for the common range and a balanced heap approach for outliers.
   - Maintain a count array for numbers in [0, 100] and use heaps for numbers outside this range.
   - Adjust median calculation accordingly, based on the distribution of numbers.

This approach ensures efficient handling of the data stream and maintains the balance required to quickly compute the median.

In [13]:
class MedianFinderRange:
    def __init__(self):
        self.counts = [0] * 101
        self.total_count = 0

    def addNum(self, num: int) -> None:
        self.counts[num] += 1
        self.total_count += 1

    def findMedian(self) -> float:
        count = 0
        median1 = None
        median2 = None
        for i in range(101):
            count += self.counts[i]
            if median1 is None and count >= (self.total_count + 1) // 2:
                median1 = i
            if median2 is None and count >= (self.total_count // 2) + 1:
                median2 = i
                break
        if self.total_count % 2 == 0 and median1 is not None and median2 is not None:
            return (median1 + median2) / 2.0
        elif median2 is not None:
            return median2

        return 0.0


# Example usage
medianFinder = MedianFinderRange()
medianFinder.addNum(1)
medianFinder.addNum(2)
print(medianFinder.findMedian())  # Output: 1.5
medianFinder.addNum(3)
print(medianFinder.findMedian())  # Output: 2.0

1.5
2
